In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler,StandardScaler


class SENET(layers.Layer):
    def __init__(self, field_size,emb_size, r):
        super(SENET, self).__init__()
        self.f = field_size
        self.m = emb_size
        self.MLP1 = layers.Dense(units=(field_size//r), activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.MLP2 = layers.Dense(units=field_size,kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu')
        
    def call(self, inputs):
        inputs = tf.reshape(inputs,shape=[-1, self.f, self.m])
        x = inputs
        x = tf.reduce_mean(x, axis=2)
        x = tf.reshape(x,shape=[-1, self.f])
        x = self.MLP1(x)
        x = self.MLP2(x)
        outputs = inputs*tf.reshape(x,shape=[-1, self.f,1])
        return tf.reshape(outputs,shape=[-1, self.f*self.m])


class ResNet(layers.Layer):
    def __init__(self, hidden_unit, dim_stack):
        super(ResNet, self).__init__()
        self.layer1 = layers.Dense(units=hidden_unit, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.layer2 = layers.Dense(units=dim_stack,kernel_regularizer=tf.keras.regularizers.l2(0.01), activation=None)

    def call(self, inputs):
        x = inputs
        x = self.layer1(x)
        x = self.layer2(x)
        outputs = x + inputs
        return outputs

class DeepFM(Model):
    def __init__(self, spare_feature_columns, dense_feature_columns, k, w_reg, v_reg, hidden_units, output_dim, activation, drop_out,Use_DNN=True,Use_Res=False):
        super(DeepFM, self).__init__()
        self.spare_feature_columns = spare_feature_columns
        self.dense_feature_columns = dense_feature_columns
        self.w_reg = w_reg
        self.v_reg = v_reg
        self.k = k
        self.Use_DNN = Use_DNN
        self.Use_Res = Use_Res

        # embedding
        self.embedding_layer = {'embed_layer{}'.format(i): layers.Embedding(feat['vocabulary_size'], self.k)
                                for i, feat in enumerate(self.spare_feature_columns)}

        # 做完embedding后的维度
        self.dense_dim = len(self.dense_feature_columns)
        self.spare_dim = len(self.spare_feature_columns)*self.k
        self.onedim = self.dense_dim + self.spare_dim 
        
        self.SENET = tf.keras.Sequential()
        self.SENET.add(SENET(len(self.spare_feature_columns), self.k, 2))
        self.deep_dim = self.onedim + self.spare_dim

        if(self.Use_Res):
            #Res
            self.DNN = tf.keras.Sequential()
            for hidden in hidden_units:
                self.DNN.add(ResNet(hidden,self.deep_dim))
                self.DNN.add(layers.BatchNormalization())
                self.DNN.add(layers.Activation(activation))
                self.DNN.add(layers.Dropout(drop_out))
            self.DNN.add(layers.Dense(output_dim, activation=None))    
        
        if(self.Use_DNN):
            # dnn
            self.DNN = tf.keras.Sequential()
            for hidden in hidden_units:
                self.DNN.add(layers.Dense(hidden, kernel_regularizer=tf.keras.regularizers.l2(0.01)))
                self.DNN.add(layers.BatchNormalization())
                self.DNN.add(layers.Activation(activation))
                self.DNN.add(layers.Dropout(drop_out))
            self.DNN.add(layers.Dense(output_dim, activation=None))

    def build(self, input_shape):
        self.b = self.add_weight(name='b', shape=(1,), initializer=tf.zeros_initializer(), trainable=True, )
        self.w = self.add_weight(name='w', shape=(self.onedim, 1), initializer=tf.random_normal_initializer(), trainable=True, regularizer=tf.keras.regularizers.l2(self.w_reg))
        self.v = self.add_weight(name='v', shape=(self.onedim, self.k), initializer=tf.random_normal_initializer(), trainable=True, regularizer=tf.keras.regularizers.l2(self.v_reg))

    def call(self, inputs, training=None, mask=None):
        # dense_inputs: 数值特征，13维
        # sparse_inputs： 类别特征，26维
        dense_inputs, sparse_inputs = inputs[:, :13], inputs[:, 13:]

        # embedding
        sparse_embed = tf.concat([self.embedding_layer['embed_layer{}'.format(i)](sparse_inputs[:, i]) for i in range(sparse_inputs.shape[1])], axis=1)  # (batchsize, 26*k)
        
        SENET_embed = self.SENET(sparse_embed)
        
        # FM、Deep 共享embedding
        FM_x = tf.concat([dense_inputs, sparse_embed], axis=1)  # (batchsize, 26*embed_dim + 13)
       
        deep_x = tf.concat([FM_x, SENET_embed], axis=1)
       

        # FM part
        linear_part = tf.matmul(FM_x, self.w) + self.b  # (batchsize, 1)
        inter_cross1 = tf.square(FM_x @ self.v)  # (batchsize, k)
        inter_cross2 = tf.matmul(tf.pow(FM_x, 2), tf.pow(self.v, 2))  # (batchsize, k)
        cross_part = 0.5 * tf.reduce_sum(inter_cross1 - inter_cross2, axis=1, keepdims=True)  # (batchsize, 1)
        fm_output = linear_part + cross_part

        # Deep part
        dnn_out = self.DNN(deep_x)  # (batchsize, 1)
        
        output = tf.nn.sigmoid(fm_output + dnn_out)
        #output = tf.nn.sigmoid(fm_output)
        
        return output


In [2]:
def sparseFeature(feat, vocabulary_size, embed_dim):
    return {'feat': feat, 'vocabulary_size': vocabulary_size, 'embed_dim': embed_dim}

def denseFeature(feat):
    return {'feat': feat}

In [3]:
import tensorflow.keras.backend as K
def Focal_Loss(y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    loss = 0.8*K.pow(1.0 - y_pred,1.0)*y_true * K.log(y_pred + 1e-10)+0.2*K.pow(y_pred,1.0)*(1.0 - y_true) * K.log(1.0 - y_pred + 1e-10)
    #loss = 0.5*y_true * K.log(y_pred + 1e-15)+0.5*(1.0 - y_true) * K.log(1.0 - y_pred + 1e-15)
    #loss = y_true * K.log(y_pred + 1e-10)+(1.0 - y_true) * K.log(1.0 - y_pred + 1e-10)
    return -K.mean(loss, axis=-1)

In [4]:
if __name__ == '__main__':

    # I1-I13：总共 13 列数值型特征
    # C1-C26：共有 26 列类别型特征
    dense_features = ['I' + str(i) for i in range(1, 14)]
    sparse_features = ['C' + str(i) for i in range(1, 27)]
    target_columns = ['label']
    columns = target_columns + dense_features + sparse_features
    data = pd.read_csv("./data/dac/train.txt",sep='\t',names = columns, nrows=200000)
    
    data[dense_features] = data[dense_features].fillna(0.0)
    data[sparse_features] = data[sparse_features].fillna('-1')
    
    #data[dense_features] = StandardScaler().fit_transform(data[dense_features])
        
    for f in dense_features:
        data[f] = data[f].apply(lambda x: np.log(x+1) if x>-1 else -1)
    data[dense_features] = MinMaxScaler().fit_transform(data[dense_features])

    for f in sparse_features:
        data[f] = LabelEncoder().fit_transform(data[f])
    
    data_X = data.iloc[:, 1:]
    data_y = data['label'].values

    dense_feature_columns = [denseFeature(feat) for feat in dense_features]
    spare_feature_columns = [sparseFeature(feat, data_X[feat].nunique(),3) for feat in sparse_features]
    
    train_X, test_X, train_y, test_y = train_test_split(data_X, data_y, test_size=0.2, random_state=0, stratify=data_y)
    
    #tmp_X, test_X, tmp_y, test_y = train_test_split(data_X, data_y, test_size=0.2, random_state=42, stratify=data_y)
    #train_X, val_X, train_y, val_y = train_test_split(tmp_X, tmp_y, test_size=0.2, random_state=42, stratify=tmp_y)

#     model = DeepFM(spare_feature_columns = spare_feature_columns,
#                    dense_feature_columns = dense_feature_columns,
#                    k = 15,
#                    w_reg = 0.01,
#                    v_reg = 0.001,
#                    hidden_units= [50,50,50,50,50],
#                    output_dim = 1,
#                    activation = 'relu',
#                    drop_out = 0.7,
#                    Use_DNN=False,
#                    Use_Res=True)

#     adam = optimizers.Adam(lr=0.005, decay=0.001)
     
#     model.compile(
#         optimizer=adam,
#         loss=Focal_Loss,
#         metrics=[metrics.AUC(), metrics.Recall()]
#     )
    
#     log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#     model.fit(
#         train_X.values, train_y,
#         validation_data=(test_X.values, test_y),
#         batch_size=2048,
#         epochs=5,
#         verbose=1,
#         callbacks=[tensorboard_callback]
#     )
    

    


In [5]:
def trian(emb_k,units,dropout,spare_column,dense_column,train_X,train_y,test_X, test_y):
    model = DeepFM(spare_feature_columns = spare_column,
                   dense_feature_columns = dense_column,
                   k = emb_k,
                   w_reg = 0.01,
                   v_reg = 0.001,
                   hidden_units= units,
                   output_dim = 1,
                   activation = 'relu',
                   drop_out = dropout,
                   Use_DNN=False,
                   Use_Res=True)
    
    adam = optimizers.Adam(lr=0.005, decay=0.001)
     
    model.compile(
        optimizer=adam,
        loss = Focal_Loss,
        #loss='binary_crossentropy',
        metrics=[metrics.AUC()]
    )
    units = [str(x) for x in units]
    log_dir="logs/last/%s_%s_layer%s" %(emb_k,dropout,"_".join(units))  
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#     if emb_k==5 or emb_k==10:
#         epoch=8
#     else:
#         epoch=5 loss="binary_crossentropy"
        
    model.fit(
        train_X.values, train_y,
        validation_data=(test_X.values, test_y),
        batch_size=2048,
        epochs=10,
        verbose=1,
        callbacks=[tensorboard_callback]
    )
    

In [6]:
for units in [[4000]]:
    for k in [24]:
        for dropout in [0.7]:
            trian(k,units,dropout,spare_feature_columns,dense_feature_columns,train_X,train_y,test_X, test_y)

Epoch 1/10
 1/79 [..............................] - ETA: 0s - loss: 38.9127 - auc: 0.4979WARNING:tensorflow:From F:\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
79/79 [==============================] - 27s 339ms/step - loss: 2.1186 - auc: 0.7126 - val_loss: 0.1272 - val_auc: 0.7674
Epoch 2/10
79/79 [==============================] - 25s 310ms/step - loss: 0.0954 - auc: 0.8329 - val_loss: 0.1249 - val_auc: 0.7738
Epoch 3/10
79/79 [==============================] - 25s 312ms/step - loss: 0.0740 - auc: 0.9144 - val_loss: 0.1408 - val_auc: 0.7568
Epoch 4/10
79/79 [==============================] - 25s 317ms/step - loss: 0.0657 - auc: 0.9409 - val_loss: 0.1373 - val_auc: 0.7361
Epoch 5/10
79/79 [==============================] - 25s 315ms/step - loss: 0.0637 - auc: 0.9512 - val_l